In [18]:
import os
import speech_recognition as sr
from pydub import AudioSegment

def prepare_voice_file(path: str) -> str:
    if os.path.splitext(path)[1] == '.wav':
        return path
    elif os.path.splitext(path)[1] in ('.mp3', '.m4a', '.ogg', '.flac'):
        audio_file = AudioSegment.from_file(
            path, format=os.path.splitext(path)[1][1:])
        wav_file = os.path.splitext(path)[0] + '.wav'
        audio_file.export(wav_file, format='wav')
        return wav_file
    else:
        raise ValueError(
            f'Unsupported audio format: {format(os.path.splitext(path)[1])}')

def segment_audio(audio_path: str, segment_length: int = 60000):
    audio = AudioSegment.from_file(audio_path)
    segments = []
    for i in range(0, len(audio), segment_length):
        segment = audio[i:i+segment_length]
        segments.append(segment)
    return segments

def transcribe_audio(audio_data, language) -> str:
    print('Transcribiendo segmento de audio...')
    r = sr.Recognizer()
    try:
        text = r.recognize_google(audio_data, language=language)
        return text
    except sr.RequestError as e:
        print(f"No se pudieron obtener resultados del servicio de reconocimiento de voz de Google; {e}")
    except sr.UnknownValueError:
        print("Google Speech Recognition no pudo entender el audio")
    except Exception as e:
        print(f"Ocurrió un error: {e}")
    return ""

def write_transcription_to_file(text, output_file) -> None:
    print('Escribiendo transcripción en archivo...')
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(text)

def speech_to_text(input_path: str, output_path: str, language: str) -> None:
    wav_file = prepare_voice_file(input_path)
    segments = segment_audio(wav_file)
    full_transcription = ""

    for i, segment in enumerate(segments):
        print(f'Procesando segmento {i+1} de {len(segments)}...')
        segment_file = f"temp_segment_{i}.wav"
        segment.export(segment_file, format="wav")

        with sr.AudioFile(segment_file) as source:
            audio_data = sr.Recognizer().record(source)
            text = transcribe_audio(audio_data, language)
            full_transcription += text + " "

        os.remove(segment_file)

    write_transcription_to_file(full_transcription.strip(), output_path)
    print('Transcripción completa:')
    print(full_transcription)

if __name__ == '__main__':
    print('Please enter the path to an audio file (WAV, MP3, M4A, OGG, or FLAC):')
    
    input_path = 'C:/git/IAClass/15_projectU4_reforma_judical_org_autonomos/assets_youtube/La_Reforma_Constitucional_a_los_Organismos_Autónomos_Parte_1.mp3'
    output_path = 'C:/git/IAClass/15_projectU4_reforma_judical_org_autonomos/data_scraped/tiktok_audio/salida_La_Reforma_Constitucional_a_los_Organismos_Autónomos_Parte_1.txt'
    language = 'es-ES'

    if not os.path.isfile(input_path):
        print('Error: Audio File not found.')
        exit(1)
    
    if os.path.isfile(output_path):
        print('Warning: Output file already exists. It will be overwritten.')
    
    try:
        print('input_path: ', input_path)
        print('output_path: ', output_path)
        print('language: ', language)
        
        speech_to_text(input_path, output_path, language)
    except Exception as e:
        print('Error:', e)
        exit(1)

Please enter the path to an audio file (WAV, MP3, M4A, OGG, or FLAC):
input_path:  C:/git/IAClass/15_projectU4_reforma_judical_org_autonomos/assets_youtube/La_Reforma_Constitucional_a_los_Organismos_Autónomos_Parte_1.mp3
output_path:  C:/git/IAClass/15_projectU4_reforma_judical_org_autonomos/data_scraped/tiktok_audio/salida_La_Reforma_Constitucional_a_los_Organismos_Autónomos_Parte_1.txt
language:  es-ES
Procesando segmento 1 de 53...
Transcribiendo segmento de audio...
Procesando segmento 2 de 53...
Transcribiendo segmento de audio...
Procesando segmento 3 de 53...
Transcribiendo segmento de audio...
Procesando segmento 4 de 53...
Transcribiendo segmento de audio...
Procesando segmento 5 de 53...
Transcribiendo segmento de audio...
Procesando segmento 6 de 53...
Transcribiendo segmento de audio...
Procesando segmento 7 de 53...
Transcribiendo segmento de audio...
Procesando segmento 8 de 53...
Transcribiendo segmento de audio...
Procesando segmento 9 de 53...
Transcribiendo segmento d

In [ ]:
import yt_dlp
from pathlib import Path

class SimpleAudioDownloader:
    def download_audio(self, url: str, output_path: str) -> dict:
        """
        Descarga el audio de un video de YouTube
        
        Args:
            url (str): URL del video de YouTube
            output_path (str): Ruta donde se guardará el audio
            
        Returns:
            dict: Resultado de la descarga con status y mensaje
        """
        try:
            # Crear el directorio de salida si no existe
            output_dir = Path(output_path)
            output_dir.mkdir(parents=True, exist_ok=True)
            
            # Configuración para la descarga
            ydl_opts = {
                'format': 'bestaudio/best',
                'postprocessors': [{
                    'key': 'FFmpegExtractAudio',
                    'preferredcodec': 'mp3',
                    'preferredquality': '192',
                }],
                'outtmpl': str(output_dir / '%(title)s.%(ext)s'),
                'quiet': False,
                'no_warnings': True
            }
            
            # Realizar la descarga
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                # Obtener información del video primero
                info = ydl.extract_info(url, download=False)
                video_title = info.get('title', 'Unknown Title')
                
                print(f"Descargando: {video_title}")
                ydl.download([url])
                
                return {
                    'status': 'success',
                    'message': f'Audio descargado exitosamente: {video_title}',
                    'title': video_title,
                    'output_path': str(output_dir)
                }
                
        except Exception as e:
            return {
                'status': 'error',
                'message': f'Error durante la descarga: {str(e)}'
            }

def main():
    # URL del video y ruta de salida
    video_url = "https://www.youtube.com/watch?v=aUIsKOrg_IY"  # Reemplaza con tu URL
    output_path = "C:/git/IAClass/15_projectU4_reforma_judical_org_autonomos/data_scraped/youtube_audio/"  # Reemplaza con tu ruta
    
    # Crear instancia del descargador y ejecutar la descarga
    downloader = SimpleAudioDownloader()
    result = downloader.download_audio(video_url, output_path)
    
    # Mostrar resultado
    if result['status'] == 'success':
        print(f"\n✅ {result['message']}")
        print(f"📁 Guardado en: {result['output_path']}")
    else:
        print(f"\n❌ {result['message']}")

if __name__ == "__main__":
    main()

[youtube] Extracting URL: https://www.youtube.com/watch?v=aUIsKOrg_IY
[youtube] aUIsKOrg_IY: Downloading webpage
[youtube] aUIsKOrg_IY: Downloading ios player API JSON
[youtube] aUIsKOrg_IY: Downloading mweb player API JSON
[youtube] aUIsKOrg_IY: Downloading m3u8 information
Descargando: México en  OSCURIDAD total tras la ELIMINACIÓN del INAI | MLDA
[youtube] Extracting URL: https://www.youtube.com/watch?v=aUIsKOrg_IY
[youtube] aUIsKOrg_IY: Downloading webpage
[youtube] aUIsKOrg_IY: Downloading ios player API JSON
[youtube] aUIsKOrg_IY: Downloading mweb player API JSON
[youtube] aUIsKOrg_IY: Downloading m3u8 information
[info] aUIsKOrg_IY: Downloading 1 format(s): 251
[download] Destination: C:\git\IAClass\15_projectU4_reforma_judical_org_autonomos\data_scraped\youtube_audio\México en  OSCURIDAD total tras la ELIMINACIÓN del INAI ｜ MLDA.webm
[download] 100% of   32.53MiB in 00:00:03 at 10.01MiB/s    
[ExtractAudio] Destination: C:\git\IAClass\15_projectU4_reforma_judical_org_autonomos\